In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from glob import glob
import yaml

In [ ]:
!pip install ultralytics
from ultralytics.data.converter import convert_coco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.1/749.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import os
from glob import glob
import yaml

# Have to move the photos from the batch folders to one folder since YOLOv8 YAML file
# does not suppport lists of files

# Path to dataset folder
dataset_path = '/content/drive/MyDrive/TACO/data'
batches = [os.path.join(dataset_path, batch) for batch in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, batch))]

# Initial value, rename since there are copies of names
value_str = '0000'

for batch_path in batches:

    images = glob(os.path.join(batch_path, '*.jpg')) + glob(os.path.join(batch_path, '*.JPG'))

    for img in images:

        # Place into main dataset folder
        os.replace(img, dataset_path+'/'+value_str+'.jpg')

        # Retrieve corresponding label file
        label_path = os.path.join(batch_path, os.path.basename(img).replace('.jpg', '.txt').replace('.JPG', '.txt'))
        # Place into main dataset folder
        os.replace(label_path, dataset_path+'/'+value_str+'.txt')

        # Count up
        value_int = int(value_str) + 1
        value_str = str(value_int).zfill(4)

print(value_str)

## YOLOv8 Configurations File 💾

In [ ]:
# Config file
config = {
    'path' : '/content/drive/MyDrive/TACO/data',
    'train': '/content/drive/MyDrive/TACO/data/train',
    'val': '/content/drive/MyDrive/TACO/data/valid',
    'test': '/content/drive/MyDrive/TACO/data/test',
    'nc': 1,  # Number of classes
    'names': ['Litter']  # Class names
}

# Save the configuration file
with open('/content/drive/MyDrive/TACO/TACO.yaml', 'w') as file:
    yaml.dump(config, file, sort_keys=False)

dataset_path = config['path']

## Splitting Data

In [ ]:
import random

In [ ]:
def split_data(images, ratio):
    """
    Splits a list of images into training, validation, and test sets.
    """
    random.shuffle(images)
    n = len(images)
    train_count = int(n * ratio['train'])
    val_count = int(n * ratio['valid'])
    # The rest goes to test
    return {
        'train': images[:train_count],
        'valid': images[train_count:train_count+val_count],
        'test': images[train_count+val_count:]
    }

In [ ]:
def move_files(files, target_dir):
    """Move files to the specified directory, placing images and labels in their subdirectories."""
    for src in files:
        # Determine the base name for creating corresponding label path
        base_name = os.path.basename(src)
        label_src = src.replace('.jpg', '.txt')

        # Construct the destination paths
        image_dest = os.path.join(target_dir, 'images', base_name)
        label_dest = os.path.join(target_dir, 'labels', base_name.replace('.jpg', '.txt'))

        # Ensure destination subdirectories exist
        os.makedirs(os.path.dirname(image_dest), exist_ok=True)
        os.makedirs(os.path.dirname(label_dest), exist_ok=True)

        # Move the image
        os.replace(src, image_dest)
        # Move the corresponding label file, if exists
        if os.path.exists(label_src):
            os.replace(label_src, label_dest)

In [ ]:
# Configuration and paths
base_dir = '/content/drive/MyDrive/TACO/data'

directories = {
    'train': os.path.join(base_dir, 'train'),
    'valid': os.path.join(base_dir, 'valid'),
    'test': os.path.join(base_dir, 'test')
}

split_ratios = {'train': 0.7, 'valid': 0.15, 'test': 0.15}

# Gather all image paths from subfolders
images = glob(os.path.join(base_dir, "*.jpg"))
images_with_labels = [img for img in images if os.path.exists(img.replace('.jpg', '.txt'))]

#Split data
data_split = split_data(images, split_ratios)

# Move files into their respective directories
for split, files in data_split.items():
    move_files(files, directories[split])

In [ ]:
# Had to replace the 'Litter' text in the labels to the class_id = 0
import os
from glob import glob

labels = glob(os.path.join('/content/drive/MyDrive/TACO/data', '**', '*.txt'), recursive=True)
print(len(labels))

for label in labels:
    with open(label, 'r') as f:
        lines = f.readlines()
    for line in lines:
        modified_lines = [line.replace('Litter', '0') if line.startswith('Litter') else line for line in lines]

    # Write the modified contents back to the file
    with open(label, 'w') as file:
        file.writelines(modified_lines)

905
